In [ ]:
import os
import gc
import h5py
import pandas as pd
import numpy as np

import sklearn.metrics as metrics

import matplotlib.pyplot as plt
plt.style.use("paper.mplstyle")
import seaborn as sns

import sys
sys.path.append("../")
import utils_plot

In [ ]:
cm = 1/2.54 
plot_alpha=0.2
h=5.8
w=6

resp_res_path = "/cluster/work/grlab/clinical/hirid2/research/event_based_analysis/resp/"
resp_fig_path = "paper_figures_resp_emer_0"
if not os.path.exists(resp_fig_path):
    os.mkdir(resp_fig_path)

In [ ]:
def read_results(vent_path,
                 pred_type, 
                 horizon_cols, 
                 param1, 
                 param2, 
                 param3, 
                 metric_bsl, 
                 metric_lgb, 
                 lst_seed,
                 fill_horizon=10):
    df_res = []
    for seed in lst_seed:
        out_dir = vent_path%seed
        for f in os.listdir(out_dir):
            if pred_type == "emergency":
                if "res_" in f and "npz" in f and "emer_" in f:
                    tmp = np.load(os.path.join(out_dir, f))
                    prediction_horizon_l = int(f.split("_")[3])
                    prediction_horizon_r = int(f.split("_")[4])
                    num_leaves = int(f.split("_")[5][1:])
                    num_rounds = int(f.split("_")[6][1:])
                    stopping_rounds = int(f.split("_")[7][1:-4])
                    dataset = f.split("_")[2]
                    df_res.append( [prediction_horizon_l, 
                                    prediction_horizon_r, 
                                    num_leaves, 
                                    num_rounds, 
                                    stopping_rounds, 
                                    dataset, 
                                    tmp["MAE"], 
                                    np.mean(np.abs(tmp["groundtruth"]-tmp["median_gt_train"])), 
                                   ] )
            elif pred_type == "in-icu":
                if "res_" in f and "npz" in f and "emer_" not in f:
                    try:
                        tmp = np.load(os.path.join(out_dir, f), allow_pickle=True)
                    except:
                        continue
                    prediction_horizon_l = int(f.split("_")[2])
                    prediction_horizon_r = int(f.split("_")[3])
                    num_leaves = int(f.split("_")[4][1:])
                    num_rounds = int(f.split("_")[5][1:])
                    stopping_rounds = int(f.split("_")[6][1:-4])
                    dataset = f.split("_")[1]
                    df_res.append( [prediction_horizon_l, 
                                    prediction_horizon_r, 
                                    num_leaves, 
                                    num_rounds, 
                                    stopping_rounds, 
                                    dataset,
                                    tmp["MAE_RF"], 
                                    tmp["MAE_baseline"]
                                   ] )

    if pred_type == "emergency":
        df_res = pd.DataFrame(df_res, columns=horizon_cols+
                                              [param1, 
                                               param2, 
                                               param3, 
                                               "Dataset", 
                                               metric_lgb,
                                               metric_bsl
                                              ])
    elif pred_type == "in-icu":
        df_res = pd.DataFrame(df_res, columns=horizon_cols+
                                              [param1, 
                                               param2,
                                               param3, 
                                               "Dataset", 
                                               metric_lgb, 
                                               metric_bsl
                                              ])

    for col in df_res.columns:
        if col=="Dataset":
            continue
        df_res.loc[:,col] = df_res[col].astype(float)    
    for p in [param1, param2, param3]:
        print(p, np.sort(df_res[p].unique()).tolist())
    dict_res = {}
    for dset in ["train", "val", "test"]:
        dict_res.update({dset: df_res[df_res.Dataset==dset].rename(columns={metric_bsl: "{} ({})".format(metric_bsl,dset), metric_lgb: "{} ({})".format(metric_lgb,dset)})})
        dict_res.update({dset: dict_res[dset].drop(columns=["Dataset"]).groupby(horizon_cols + [param1, param2, param3]).mean()})

    df_best = dict_res["val"].reset_index().sort_values(horizon_cols+["{} (val)".format(metric_lgb)]).drop_duplicates(horizon_cols, keep="first")
    df_best = df_best.set_index(horizon_cols + [param1, param2, param3])
    df_best = pd.concat([df_best, 
                         dict_res["test"], 
                         dict_res["train"]], join="inner", axis=1)
    return df_best

In [ ]:
vent_path = "/cluster/work/grlab/clinical/hirid2/research/vent_planning"

horizon_cols = ["PredictionHorizon_l", "PredictionHorizon_r"]
metric_bsl = "Baseline"
metric_lgb = "RMS"

param1 = "Num_leaves"
param2 = "Num_rounds"
param3 = "Stopping_rounds"

lst_seed = np.arange(2020,2025)

pred_type = "in-icu"
pred_type = "emergency"

is_kanonym = True

In [ ]:
df_best_emer = []
df_best_icu = []
for i in np.arange(1,6):
    out_dir = "res_%%d_rms_new_temporal_%d"%i
    df_best_emer.append(read_results(os.path.join(vent_path, out_dir), "emergency", horizon_cols, param1, param2, param3, metric_bsl, metric_lgb, lst_seed))
    df_best_icu.append(read_results(os.path.join(vent_path, out_dir), "in-icu", horizon_cols, param1, param2, param3, metric_bsl, metric_lgb, lst_seed))

In [ ]:
n = 0
for title, df in [("In-ICU patient ventilation prediction", df_best_icu), 
                  ("Newly admitted ventilated emergency patients", df_best_emer)]:
    df_melt = []
    for k, df_split in enumerate(df):
        tmp = df_split.reset_index()
        tmp.loc[:,"Prediction Horizon"] = ["[{:.0f},{:.0f}] h".format(tmp.iloc[i][horizon_cols[0]], tmp.iloc[i][horizon_cols[1]]) for i in range(len(tmp))]
        tmp = tmp.melt(id_vars=["Prediction Horizon"],
                       value_vars=["{} (test)".format(metric_bsl), "{} (test)".format(metric_lgb)])
        tmp = tmp.rename(columns={"variable": "Method", "value": "MAE"})
        tmp = tmp.replace("{} (test)".format(metric_bsl), metric_bsl)
        tmp = tmp.replace("{} (test)".format(metric_lgb), metric_lgb)
        tmp.loc[:,"temporal_split"] = k
        df_melt.append(tmp)
    df_melt = pd.concat(df_melt).reset_index(drop=True)

    plt.figure(figsize=(w*cm,h*0.7*cm))
    sns.barplot(x="Prediction Horizon", y="MAE", hue="Method", data=df_melt, zorder=2)
    plt.xticks(rotation=30, horizontalalignment="right")
    plt.grid(axis='y')
    plt.title(title)
    plt.savefig(os.path.join(resp_fig_path, "fig5_subfig%d"%n))
    n+=1

In [ ]:
all_err_weekday = []
rug_plot = []

rug_plot_sign = []
for i in np.arange(1,6):

    for j in range(len(df_best_icu[i-1])):
        df_err_weekday = []
        rug_plot_tmp = []
        feat_dir = "resource_planning_features_rms_new_temporal_%d"%i
        
        for seed in np.arange(2020,2025):
            out_dir = "res_%d_rms_new_temporal_%d"%(seed, i)
            
            tmp = df_best_icu[i-1].index[j]
            try:
                res_icu= np.load(os.path.join(vent_path, out_dir, 'res_test_%d_%d_l%d_r%d_s%d.npz'%(tmp[0],tmp[1],tmp[2],tmp[3],tmp[4])))
            except:
                continue
            
            tmp = df_best_emer[i-1].index[j]
            try:
                res_emer= np.load(os.path.join(vent_path, out_dir, 'res_emer_test_%d_%d_l%d_r%d_s%d.npz'%(tmp[0],tmp[1],tmp[2],tmp[3],tmp[4])))
            except:
                continue
                
            lbl_test = pd.read_hdf(os.path.join(vent_path, feat_dir, "label_test_%d_%d.h5"%(tmp[0],tmp[1])))
            lbl_test = lbl_test.loc[res_emer['lst_dt']]
            
            ts_icu = np.stack([res_icu['groundtruth'],res_icu['prediction'], res_icu['prediction_ffill']], axis=1)
            ts_icu = pd.DataFrame(ts_icu, columns=['Groundtruth (in-ICU)', 'RMS (in-ICU)', 'Baseline (in-ICU)'])
            ts_icu.loc[:,'Datetime'] = res_icu['lst_dt']
            ts_emer = np.stack([res_emer['groundtruth'],res_emer['prediction']], axis=1)
            ts_emer = pd.DataFrame(ts_emer,columns=['Groundtruth (Emergency)', 'RMS (Emergency)'])
            ts_emer.loc[:,'Datetime'] = res_emer['lst_dt']
            ts_emer.loc[:,'Baseline (Emergency)'] = res_emer['median_gt_train']

            ts_aggr = ts_icu.merge(ts_emer, left_on='Datetime', right_on='Datetime')
            ts_aggr = ts_aggr.merge(lbl_test[["VentUse_elective_future", "VentUse_future"]], left_on="Datetime", right_index=True)
            
#             ts_aggr['Groundtruth (all)'] = ts_aggr["VentUse_future"] - ts_aggr['VentUse_elective_future'].fillna(0)
#             ts_aggr['RMS (all)'] = ts_aggr['RMS (in-ICU)'] + ts_aggr['RMS (Emergency)']
            
            ts_aggr['Groundtruth (all)'] = ts_aggr["VentUse_future"]
            ts_aggr['RMS (all)'] = ts_aggr['RMS (in-ICU)'] + ts_aggr['RMS (Emergency)'] + ts_aggr['VentUse_elective_future'].fillna(0)
            
            # ts_aggr['Baseline (all)'] = ts_aggr['Baseline (in-ICU)'] + ts_aggr['Baseline (Emergency)']
            ts_aggr['Baseline (all)'] = ts_aggr['Baseline (in-ICU)'] #emer_patient_num

            ts_aggr.loc[:,'Weekday'] =  ts_aggr.Datetime.dt.weekday
            ts_aggr.loc[:,'Error (RMS %d)'%seed] = np.abs(ts_aggr['Groundtruth (all)'] - np.round(ts_aggr['RMS (all)']))
            ts_aggr.loc[:,'Error (Baseline %d)'%seed] = np.abs(ts_aggr['Groundtruth (all)'] - ts_aggr['Baseline (all)'])
            
            ts_aggr.loc[:,'Error with sign (RMS %d)'%seed] = (np.round(ts_aggr['RMS (all)']) - ts_aggr['Groundtruth (all)'])
            ts_aggr.loc[:,'Error with sign (Baseline %d)'%seed] = (ts_aggr['Baseline (all)'] - ts_aggr['Groundtruth (all)'])
            
            
            df_err_weekday.append(ts_aggr[['Weekday', 'Error (RMS %d)'%seed]].groupby('Weekday').mean())
            
            rug_plot_tmp2 = ts_aggr.copy()
            rug_plot_tmp2 = rug_plot_tmp2.rename(columns={'Groundtruth (all)':'Groundtruth (%d)'%seed,
                                                          'RMS (all)': 'RMS (%d)'%seed,
                                                          'Baseline (all)': 'Baseline (%d)'%seed})
            
            rug_plot_tmp.append(rug_plot_tmp2[['Datetime','Groundtruth (%d)'%seed, 'RMS (%d)'%seed, 'Baseline (%d)'%seed, 'Error (RMS %d)'%seed,'Error (Baseline %d)'%seed]].set_index('Datetime') )
            rug_plot_tmp3 = rug_plot_tmp2.copy()
            rug_plot_tmp3.loc[:,"Prediction Horizon"] = "[%d,%d]h"%(tmp[0], tmp[1])
            rug_plot_sign.append(rug_plot_tmp3)
        if len(df_err_weekday) == 0:
            continue
        df_err_weekday = pd.concat(df_err_weekday, axis=1)
        df_err_weekday.loc[:, 'MAE (RMS)'] =  df_err_weekday[[col for col in df_err_weekday.columns if 'RMS' in col]].mean(axis=1)
        df_err_weekday.loc[:, "Prediction Horizon"] = "[%d,%d]h"%(tmp[0], tmp[1])
        df_err_weekday = df_err_weekday[['Prediction Horizon', 'MAE (RMS)']]
        df_err_weekday.loc[:,'Split'] = 'temporal_%d'%i
        df_err_weekday = df_err_weekday.reset_index()
        all_err_weekday.append(df_err_weekday)
        
        rug_plot_tmp = pd.concat(rug_plot_tmp, axis=1).reset_index()
        rug_plot_tmp.loc[:, 'Absolute Error (RMS)'] =  np.round(rug_plot_tmp[[col for col in rug_plot_tmp.columns if 'Error (RMS' in col]].mean(axis=1))
        rug_plot_tmp.loc[:, 'Absolute Error (Baseline)'] =  rug_plot_tmp[[col for col in rug_plot_tmp.columns if 'Error (Baseline' in col]].mean(axis=1)
        rug_plot_tmp.loc[:, 'Groundtruth (all)'] =  np.round(rug_plot_tmp[[col for col in rug_plot_tmp.columns if 'Groundtruth (' in col]].mean(axis=1))
        rug_plot_tmp.loc[:, 'RMS (all)'] =  rug_plot_tmp[[col for col in rug_plot_tmp.columns if 'RMS (' in col]].mean(axis=1)
        rug_plot_tmp.loc[:, 'Baseline (all)'] =  rug_plot_tmp[[col for col in rug_plot_tmp.columns if 'Baseline (' in col]].mean(axis=1)
        rug_plot_tmp.loc[:, "Prediction Horizon"] = "[%d,%d]h"%(tmp[0], tmp[1])
        rug_plot_tmp = rug_plot_tmp[['Datetime','Prediction Horizon', 'Groundtruth (all)', 'RMS (all)', 'Baseline (all)', 'Absolute Error (RMS)', 'Absolute Error (Baseline)']]
        rug_plot_tmp.loc[:, 'Split'] = 'temporal_%d'%i
        
        rug_plot.append(rug_plot_tmp)
        
all_err_weekday = pd.concat(all_err_weekday).reset_index(drop=True)
rug_plot = pd.concat(rug_plot).reset_index(drop=True)
rug_plot_sign = pd.concat(rug_plot_sign).reset_index(drop=True)

In [ ]:
df_icu_bed = pd.read_hdf("/cluster/work/grlab/clinical/hirid2/research/vent_planning/resource_planning_features_rms_new_temporal_1_icu_bed/icu_bed_test_8_16.h5")

In [ ]:
for col in rug_plot_sign.columns:
    if 'sign' in col and 'Baseline' in col:
        rug_plot_sign.loc[:,col] = np.sign(rug_plot_sign[col]).astype(float).replace(1,'Over-estimate').replace(-1,'Under-estimate').replace(0,'Correct estimate')

In [ ]:
for col in rug_plot_sign.columns:
    if 'sign' in col and 'Baseline' in col:
        plt.figure(figsize=(w*cm, h*cm))
        sns.countplot(data=rug_plot_sign, x='Prediction Horizon', hue=col, hue_order=["Under-estimate", "Over-estimate", "Correct estimate"])
        plt.tight_layout()
        plt.savefig(os.path.join(resp_fig_path, 'estimation_status'))
        plt.show()
        break

In [ ]:
delta_summary = []
for i in np.arange(1,6):
    rug_plot_tmp = rug_plot[rug_plot.Split=='temporal_%d'%i].copy()
    
    for j in range(len(df_best_icu[i-1])):
        if j==3:
            plt.figure(figsize=(w*1.5*cm, h*cm))
        tmp = df_best_icu[i-1].index[j]
        rug_plot_tmp2 = rug_plot_tmp[rug_plot_tmp["Prediction Horizon"]=="[%d,%d]h"%(tmp[0], tmp[1])].copy()
        rug_plot_tmp2.loc[:,'Diff'] = rug_plot_tmp2['Absolute Error (RMS)'] - rug_plot_tmp2['Absolute Error (Baseline)'] 
        if j==3:
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['Groundtruth (all)'], color='C7')
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['RMS (all)'], color='C6')
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['Baseline (all)'], color='C8')
            
        # sns.rugplot(data=rug_plot_tmp2, x= 'Datetime' , hue='Model Comparison', hue_order=['rms predicts better', 'Baseline predicts better'],expand_margins=False)
        for delta in range(0,4):
            rug_plot_tmp2.loc[:,'Model Comparison'] = rug_plot_tmp2.Diff.apply(lambda x: 'rms predicts better' if x<-delta else ('Baseline predicts better' if x>delta else 'equally better'))
            cnt_model_comp =rug_plot_tmp2['Model Comparison'].value_counts()
            if 'Baseline predicts better' not in cnt_model_comp.index:
                cnt_model_comp.loc['Baseline predicts better'] = 0
            if j==3:
                plt.vlines(rug_plot_tmp2.Datetime[rug_plot_tmp2['Model Comparison']=='rms predicts better'], ymax=-3*(delta-1+1), ymin=-3*(delta+1)+0.1, color='C0', linewidth=8.5)
                plt.vlines(rug_plot_tmp2.Datetime[rug_plot_tmp2['Model Comparison']=='Baseline predicts better'], ymax=-3*(delta-1+1), ymin=-3*(delta+1)+0.1, color='C1', linewidth=8.5)
                delta_summary.append([delta+1, i, cnt_model_comp.loc['rms predicts better']/len(rug_plot_tmp2)*100,cnt_model_comp.loc['Baseline predicts better']/len(rug_plot_tmp2)*100])
        if j==3:
            plt.axhline(0, linewidth=1, color='k')
            plt.axhline(-3, linewidth=1, color='k')
            plt.axhline(-6, linewidth=1, color='k')
            plt.axhline(-9, linewidth=1, color='k')
            plt.axhline(-12, linewidth=1, color='k')
            ax = plt.gca()

            plt.legend(['Groundtruth', 'RMS', 'Baseline', 'RMS predicts at least $\\delta$ better', 'Baseline predicts at least $\\delta$ better'],
                        bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                      ncols=2, mode="expand", borderaxespad=0.)            
            x_span = [np.datetime64('2018-11-01'),np.datetime64('2018-11-02')]
            y_span = rug_plot_tmp2[(rug_plot_tmp2['Datetime']<=x_span[1])&(rug_plot_tmp2['Datetime']>=x_span[0])][['Groundtruth (all)', 'RMS (all)', 'Baseline (all)']].max(axis=1).max()
            plt.xlim(x_span)
            plt.ylim([-12.3,y_span])
            plt.yticks(np.concatenate([[-12,-9,-6,-3], np.arange(0,y_span+5,5)]),
                       ['$\\delta=%d$'%y for y in range(1,5)[::-1]]+['%d'%y for y in np.arange(0,y_span+5,5)])
            ax.spines[['bottom']].set_visible(False)
            xtickslabel = np.datetime64('2018-11-01')+np.array([np.timedelta64(i,'h') for i in np.arange(0,24)])
            plt.xticks(xtickslabel, ["%0d:00"%x for x in np.arange(24)], rotation=45, horizontalalignment="right")
            plt.xlabel('2018-11-01')
            plt.tight_layout()
            plt.savefig(os.path.join(resp_fig_path, "fig5_subfig9"))
            plt.show()

delta_summary = pd.DataFrame(delta_summary, columns=['delta', 'temporal_split', 'RMS Better', 'Baseline Better'])
delta_summary = delta_summary.groupby('delta').mean().iloc[:,1:].reset_index()
delta_summary.loc[:,'RMS Better'] = delta_summary['RMS Better'].apply(lambda x: '%1.1f%%'%x)
delta_summary.loc[:,'Baseline Better'] = delta_summary['Baseline Better'].apply(lambda x: '%1.1f%%'%x)
delta_summary.loc[:,'delta'] = delta_summary['delta'].apply(lambda x: '$\delta=%d$'%x)

In [ ]:
plt.figure(figsize=(w*0.75*cm, h*cm))
the_table = plt.table(cellText=delta_summary.iloc[:,1:].values,
                      rowLabels=delta_summary['delta'].values,
                      colLabels=delta_summary.columns[1:].values,
                      loc='center')
ax = plt.gca()
ax.spines[['left','bottom']].set_visible(False)
the_table.auto_set_font_size(False)
the_table.set_fontsize(6)
plt.xticks([],[])
plt.yticks([],[])
plt.tight_layout()
plt.savefig(os.path.join(resp_fig_path, "fig5_subfig10"))

In [ ]:
for i in range(5):
    print("=====")    
    display(df_best_icu[i])
    display(df_best_emer[i])
    print("=====")

In [ ]:
err_day_aggr = []
for i in np.arange(1,6):
    rug_plot_tmp = rug_plot[rug_plot.Split=="temporal_%d"%i].copy()
    for j in range(len(df_best_icu[i-1])):
        tmp = df_best_icu[i-1].index[j]
        rug_plot_tmp2 = rug_plot_tmp[rug_plot_tmp["Prediction Horizon"]=="[%d,%d]h"%(tmp[0], tmp[1])].copy()
        rug_plot_tmp2.loc[:,'Hour'] = rug_plot_tmp2.Datetime.dt.hour
        err_day = rug_plot_tmp2[['Hour', 'Absolute Error (RMS)', 'Absolute Error (Baseline)']].groupby('Hour').mean().reset_index()
        err_day.loc[:,"Prediction Horizon"] = "[%d,%d]h"%(tmp[0], tmp[1])
        err_day.loc[:,"Split"] = "temporal_%d"%i
        err_day_aggr.append(err_day)
        
err_day = pd.concat(err_day_aggr).reset_index(drop=True)
for j in range(len(df_best_icu[i-1])):
    tmp = df_best_icu[i-1].index[j]
    err_day_tmp = err_day[err_day["Prediction Horizon"]=="[%d,%d]h"%(tmp[0], tmp[1])].copy()
    
    err_day_mean = err_day_tmp[['Hour', 'Absolute Error (RMS)', 'Absolute Error (Baseline)']].groupby("Hour").mean()
    err_day_std = err_day_tmp[['Hour', 'Absolute Error (RMS)', 'Absolute Error (Baseline)']].groupby("Hour").std()

    plt.figure(figsize=(w*cm,h*cm))
    plt.plot(err_day_mean.index, err_day_mean["Absolute Error (RMS)"], label='RMS', marker='.', color='C0')
    plt.plot(err_day_mean.index, err_day_mean["Absolute Error (Baseline)"], label='Baseline', marker='.', color='C1')
    # plt.title(err_day_tmp.iloc[0]["Prediction Horizon"])
    plt.ylabel('MAE')
    plt.xlabel('Prediction Horizon')
    xtickslabel = []
    for k in np.arange(0,24,4):
        if k+tmp[0] >= 24:
            xtickslabel.append("%02d:00$_{(+1~\\mathrm{day})}$-%02d:00$_{(+1~\\mathrm{day})}$"%((k+tmp[0])%24, (k+tmp[1])%24))
        elif k+tmp[1] >=24:
            xtickslabel.append("%02d:00-%02d:00$_{(+1~\\mathrm{day})}$"%(k+tmp[0], (k+tmp[1])%24))
        else:
            xtickslabel.append("%02d:00-%02d:00"%(k+tmp[0], k+tmp[1]))
    plt.xticks(np.arange(0,24,4), xtickslabel, rotation=90, horizontalalignment="center")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(resp_fig_path, 'fig5_subfig2%d'%j))
    plt.show()


In [ ]:
for i in np.arange(1,6):
    rug_plot_tmp = rug_plot[rug_plot.Split=='temporal_%d'%i].copy()
    for j in range(len(df_best_icu[i-1])):
        tmp = df_best_icu[i-1].index[j]
        rug_plot_tmp2 = rug_plot_tmp[rug_plot_tmp["Prediction Horizon"]=="[%d,%d]h"%(tmp[0], tmp[1])].copy()
        lbl_test = pd.read_hdf(os.path.join(vent_path, feat_dir, "label_test_%d_%d.h5"%(tmp[0],tmp[1])))
        lbl_test = lbl_test.loc[res_emer['lst_dt']]
        rug_plot_tmp2 = rug_plot_tmp2.merge(lbl_test[['VentUse']], right_index=True, left_on='Datetime')
        rug_plot_tmp2.loc[:,'Date'] = rug_plot_tmp2.Datetime.dt.date
        plt.figure(figsize=(w*cm, h*0.75*cm))
        date_summary = rug_plot_tmp2[['Date', 'VentUse']].groupby('Date').max()
        plt.hist(date_summary['VentUse'], bins=range(int(date_summary['VentUse'].max())+1), density=True, color='C7')
        plt.ylabel('Frequency')
        plt.xlabel('# ventilators in Use (Daily)')
        # plt.title("[%d,%d]h"%(tmp[0], tmp[1]))
        plt.tight_layout()
        plt.savefig(os.path.join(resp_fig_path, "fig5_subfig11"))
        plt.show()
        break
    break

In [ ]:
for dset in ["train", "val", "test"]:
    lbl_test = pd.read_hdf(os.path.join(vent_path, feat_dir, "label_%s_%d_%d.h5"%(dset, tmp[0],tmp[1])))
    print(lbl_test.index[0], lbl_test.index[-1])
    print(lbl_test.max())

In [ ]:
lst_weekday = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
all_err_weekday.loc[:,'Weekday'] = all_err_weekday['Weekday'].apply(lambda x: lst_weekday[x])

In [ ]:
plt.figure(figsize=(w*2*cm, h*0.75*cm))
sns.barplot(y='MAE (RMS)', x='Prediction Horizon', hue='Weekday', data=all_err_weekday)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
plt.savefig(os.path.join(resp_fig_path, "fig5_subfig2"))
plt.show()

In [ ]:
overall_mae = rug_plot[['Prediction Horizon', 'Split', 'Absolute Error (RMS)', 'Absolute Error (Baseline)']].groupby(['Prediction Horizon', 'Split']).mean().reset_index()
overall_mae = overall_mae[['Prediction Horizon', 'Absolute Error (RMS)', 'Absolute Error (Baseline)']].groupby('Prediction Horizon').mean()
overall_mae = overall_mae.reset_index().melt(id_vars='Prediction Horizon', var_name='Method', value_name='MAE')
overall_mae.loc[:,'Method'] = overall_mae['Method'].replace({'Absolute Error (RMS)':'RMS','Absolute Error (Baseline)':'Baseline'})
plt.figure(figsize=(w*cm,h*0.7*cm))
sns.barplot(x="Prediction Horizon", y="MAE", hue="Method", data=overall_mae, zorder=2, 
            order=['[4,8]h', '[4,12]h', '[8,12]h', '[8,16]h', '[16,24]h'])
plt.xticks(range(5), ['4-8h', '4-12h', '8-12h', '8-16h', '16-24h'], rotation=30, horizontalalignment="right")
plt.grid(axis='y')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                      ncols=2, mode="expand", borderaxespad=0.)
plt.savefig(os.path.join(resp_fig_path, "fig5_subfig12"))

In [ ]:
for i in np.arange(1,6):
    rug_plot_tmp = rug_plot[rug_plot.Split=='temporal_%d'%i].copy()
    
    for j in range(len(df_best_icu[i-1]))[-1:]:
        print('temporal_%d'%i)
        if i==5:
            plt.figure(figsize=(w*1.5*cm, h*cm))
        tmp = df_best_icu[i-1].index[j]
        rug_plot_tmp2 = rug_plot_tmp[rug_plot_tmp["Prediction Horizon"]=="[%d,%d]h"%(tmp[0], tmp[1])].copy()
        rug_plot_tmp2.loc[:,'Diff'] = rug_plot_tmp2['Absolute Error (RMS)'] - rug_plot_tmp2['Absolute Error (Baseline)'] 
        if i==5:
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['Groundtruth (all)'], color='C7')
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['RMS (all)'], color='C6')
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['Baseline (all)'], color='C8')
            

In [ ]:
for i in np.arange(1,6):
    rug_plot_tmp = rug_plot[rug_plot.Split=='temporal_%d'%i].copy()
    
    for j in range(len(df_best_icu[i-1]))[-1:]:
        print('temporal_%d'%i)
        if i==5:
            plt.figure(figsize=(w*2.75*cm, h*0.6*cm))
        tmp = df_best_icu[i-1].index[j]
        rug_plot_tmp2 = rug_plot_tmp[rug_plot_tmp["Prediction Horizon"]=="[%d,%d]h"%(tmp[0], tmp[1])].copy()
        rug_plot_tmp2.loc[:,'Diff'] = rug_plot_tmp2['Absolute Error (RMS)'] - rug_plot_tmp2['Absolute Error (Baseline)'] 
        if i==5:
            plt.yticks([],[])
            ax=plt.gca()
            ax.spines["left"].set_visible(False)
            ax = plt.gca().twinx()
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['Groundtruth (all)'], color='C7', linestyle=":", label="Actual number of mechanically ventilated patients in [8,16]h")
            plt.plot(rug_plot_tmp2.Datetime, rug_plot_tmp2['RMS (all)'], color='C6', label="Estimated number of mechanically ventilated patients in [8,16]h")
            ax.spines["left"].set_visible(False)
            ax.spines["right"].set_visible(True)
            ax.spines['right'].set_color('C6')
            ax = plt.gca()
            ax.tick_params(axis='y', colors='C6')  
            ax.yaxis.label.set_color('C6')
            plt.yticks()
            plt.xticks([],[])
            plt.xlim([np.datetime64("2019-01-01"),np.datetime64("2019-03-31")])
            plt.ylabel("Number of\nVentilated Patients")
            plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left', ncol=1, mode="expand", borderaxespad=0.)
            plt.tight_layout()
            plt.savefig("paper_figures_resp/esti_vent_long", bbox_inches="tight")

In [ ]:
for i in np.arange(1,6):
    rug_plot_tmp = rug_plot[rug_plot.Split=="temporal_%d"%i].copy()
    for j in range(len(df_best_icu[i-1])):
        tmp = df_best_icu[i-1].index[j]
        rug_plot_tmp2 = rug_plot_tmp[rug_plot_tmp["Prediction Horizon"]=="[%d,%d]h"%(tmp[0], tmp[1])].copy()
        lbl_test = pd.read_hdf(os.path.join(vent_path, feat_dir, "label_test_%d_%d.h5"%(tmp[0],tmp[1])))
        lbl_test = lbl_test.loc[res_emer['lst_dt']]
        rug_plot_tmp2 = rug_plot_tmp2.merge(lbl_test[['VentUse']], right_index=True, left_on='Datetime')

        plt.figure(figsize=(w*cm, h*0.75*cm))
        err_hist = []
        cnt = []
        xvals = np.arange(rug_plot_tmp2['VentUse'].min(),rug_plot_tmp2['VentUse'].max(),4)
        for k in xvals:
            rug_plot_tmp3 = rug_plot_tmp2[(rug_plot_tmp2['VentUse']>=k)&(rug_plot_tmp2['VentUse']<k+4)]
            cnt.append(len(rug_plot_tmp3))
            err_rms_tmp = rug_plot_tmp3['Absolute Error (RMS)'].mean()
            err_hist.append(err_rms_tmp)
        plt.bar(np.arange(len(err_hist))-0.3+0.1, err_hist, width=0.4, alpha=1, label='RMS', color='C0')
        err_hist = []
        for k in xvals:
            rug_plot_tmp3 = rug_plot_tmp2[(rug_plot_tmp2['VentUse']>=k)&(rug_plot_tmp2['VentUse']<k+4)]
            err_rms_tmp = rug_plot_tmp3['Absolute Error (Baseline)'].mean()
            err_hist.append(err_rms_tmp)
        plt.bar(np.arange(len(err_hist))+0.2, err_hist, width=0.4, alpha=1, label='Baseline', color='C1')
        plt.xticks(range(len(err_hist)), 
                  ["%d-%d\n(n=%d)"%(xvals[i],xvals[i]+4-1, cnt[i]) for i in range(len(xvals))], rotation=30, horizontalalignment='right')
        
        plt.ylabel("MAE of Ventilation\nPrediction in %d-%dh"%(tmp[0], tmp[1]))
        plt.xlabel("# Ventilators in Use (Hourly)")
        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                      ncols=2, mode="expand", borderaxespad=0.)
        plt.tight_layout()
        plt.savefig(os.path.join(resp_fig_path, "fig5_subfig3%d"%j))
        plt.show()
    break

In [ ]:
feature_cols = ["Prediction_vent", "Prediction_extu", "Prediction_fail", "Prediction_exfa", "VentUse", "weekday", "total_vent_use", "total_patient", "idx_wind"]
elective_pids = pd.read_hdf("/cluster/work/grlab/clinical/hirid2/research/1a_hdf5_clean/v8/static.h5")[["PatientID", "Emergency"]]
elective_pids = np.sort(elective_pids[elective_pids.Emergency==0].PatientID.unique())

lst_features = np.concatenate([ ["min_%s"%col for col in feature_cols[:4]], 
                                ["p25_%s"%col for col in feature_cols[:4]],
                                ["median_%s"%col for col in feature_cols[:4]],
                                ["p75_%s"%col for col in feature_cols[:4]],
                                ["max_%s"%col for col in feature_cols[:4]],
                                ["current_%s"%col for col in feature_cols[:4]],
                                ["vent_last_hour", "vent_since_admission"],
                                ["min_total_vent", "max_total_vent"],
                                ["min_total_patient", "max_total_patient"],
                                ["weekday", "hourOftheDay","timeSinceAdmission"]])

In [ ]:
lst_features  = np.concatenate((lst_features, ["curr_vent_state"]))

In [ ]:
pd.DataFrame(lst_features.reshape((-1,1)),columns=["Feature"]).to_csv("resource_planning_feature.csv", index=False)